In [212]:
import pandas as pd
import numpy as np

from datetime import datetime
import pickle
import time
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import operator
import math

import vk

%matplotlib inline

In [10]:
pd.set_option('max_columns', 100)

## Tokens

In [ ]:
vk_token = ''

## Data - загрузим данные об отзывах ЛУПов 

In [252]:
data = pd.read_csv('data/data.csv', encoding='cp1251', sep=';')
data.head(3)

,lat,long,place_id,place_name,place_website,place_rating,review_mean,review_std,checkins,users,place_website_4q
0,55.664012,37.522861,ChIJxUJiGhlNtUYRjyucIeHbeVI,"Родильный дом №4, филиал №1 ГКБ им. В.В. Виног...",http://www.roddom4.ru/,4.2,NaN,NaN,NaN,NaN,NaN
1,55.756778,37.638748,ChIJB437ifVKtUYR7EMUU6VApLg,Национальный медицинский исследовательский цен...,http://www.gnicpm.ru/,4.3,0.25,0.433013,67.0,31.0,https://www.gnicpm.ru
2,55.755245,37.656547,ChIJ2acH14xKtUYRCFp01z3Cgbk,МедЦентрСервис,http://medtsentrservis.ru/,4.7,NaN,NaN,NaN,NaN,NaN


## VK - найдем данные о группах ЛУПов

In [207]:
session = vk.Session(access_token=vk_token)
vk_api = vk.API(session)

In [214]:
for i, row in data.iterrows():
    r = vk_api.groups.search(q=row['place_name'], city_id=1, country_id = 1, count = 1)
    
    if r[0] != 0:
        
        # group id
        group_id = r[1]['gid']
        data.loc[i, 'group_id'] = group_id
        
        # count
        data.loc[i, 'group_followers_count'] =  vk_api.groups.getMembers(group_id=group_id)['count']
        
    time.sleep(0.5)     

## Alexa - посмотрим активность веб-сайтов ЛУПов

In [248]:
def parse_alexa(website = 'example.ru'):
    
    '''
    
    Функция для загрузки статистики об активности веб-сайтов ЛУПов
    
    param website: имя сайта
    
    '''
    
    url = 'https://alexa.com/siteinfo/' + str(website)
    headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
              }
    
    r = requests.get(url, headers = headers)
    
    soup = BeautifulSoup(r.text.encode('utf8'), "lxml")
    if len(soup.findAll('section', {'class': 'note-no-data'})) != 1:
        ranks = soup.findAll('div', {'class': 'rank-row'})
        
        if len(ranks[0].findAll('div')) > 0:
            world_rank = ranks[0].findAll('div')[0].findAll('strong')[0].text
            world_rank = int(''.join(x for x in world_rank if x.isdigit()))
        else:
            world_rank = None

        if len(ranks[1].findAll('div')) > 0:
            country_rank = ranks[1].findAll('div')[0].findAll('strong')[0].text
            country_rank = int(''.join(x for x in country_rank if x.isdigit()))
        else:
            country_rank = None


        link_numbers = soup.findAll('div', {'class': 'box1 box1-med3'})
        link_numbers = link_numbers[0].findAll('span')[0].text
        link_numbers = int(''.join(x for x in link_numbers if x.isdigit()))
    else: 
        country_rank, world_rank, link_numbers = None, None, None
    
    return world_rank, country_rank, link_numbers

In [249]:
# загрузим статистику веб-сайтов ЛУПа

for i, row in data.iterrows():
    web = row.place_website
    
    if pd.isnull(web):
        web = row.place_website_4q
        
    if pd.isnull(web):
        continue
        
    res = parse_alexa(web.replace('.php', ''))
    
    if res is not None:
        
        world_rank, country_rank, link_numbers = res
        
        data.loc[i, 'world_rank'] = world_rank
        data.loc[i, 'country_rank'] = country_rank
        data.loc[i, 'link_numbers'] = link_numbers
        
    time.sleep(0.5)     

In [250]:
data.head(3)

,lat,long,place_id,place_name,place_website,place_rating,review_mean,review_std,checkins,users,place_website_4q,group_id,group_followers_count,world_rank,country_rank,link_numbers
0,55.664012,37.522861,ChIJxUJiGhlNtUYRjyucIeHbeVI,"Родильный дом №4, филиал №1 ГКБ им. В.В. Виног...",http://www.roddom4.ru/,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1486796.0,102707.0,65.0
1,55.756778,37.638748,ChIJB437ifVKtUYR7EMUU6VApLg,Национальный медицинский исследовательский цен...,http://www.gnicpm.ru/,4.3,0.25,0.433013,67.0,31.0,https://www.gnicpm.ru,NaN,NaN,556465.0,38222.0,281.0
2,55.755245,37.656547,ChIJ2acH14xKtUYRCFp01z3Cgbk,МедЦентрСервис,http://medtsentrservis.ru/,4.7,NaN,NaN,NaN,NaN,NaN,100098038.0,474.0,10188597.0,NaN,0.0


## Geolocations - получим данные из геолокации ЛУПов

In [254]:
def measure(lat1, lon1, lat2, lon2):
    
    '''
    
    Функция для рассчета расстояния между двумя точками на карте
    
    '''
    
    R = 6378.137
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * \
        math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon / 2)
        
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000

In [265]:
# рассчитаем расстояние до метро и район города ЛУПов

for index, row in data.iterrows():
    
    lat = float(row.lat)
    long = float(row.long)
    
    # метро
    r = requests.get('https://geocode-maps.yandex.ru/1.x/?format=json&geocode=' + ','.join([str(lat), str(long)]) + '&kind=metro')
    r.enconding = 'cp1251'
    j = json.loads(r.text)
    ar = j.get('response').get('GeoObjectCollection').get('featureMember')
    if len(ar) != 0:
        p_lat, p_long = ar[0].get('GeoObject').get('Point').get('pos').split(' ')
        p_long, p_lat = float(p_long), float(p_lat)
        
        data.loc[index, 'distance_metro'] = measure(lat, long, p_lat, p_long)

        
    # район
    r = requests.get('https://geocode-maps.yandex.ru/1.x/?format=json&geocode=' + ','.join([str(lat), str(long)]) + '&kind=district' + '&spn=0.552069,0.400552')
    r.enconding = 'cp1251'
    j = json.loads(r.text)
    ar = j.get('response').get('GeoObjectCollection').get('featureMember')
    if len(ar) != 0 and ar[0] is not None:
        try: 
            data.loc[index, 'district'] = ar[0].get('GeoObject').get('metaDataProperty').get('GeocoderMetaData').get('text')
        except AttributeError:
            continue

In [266]:
data.head(3)

,lat,long,place_id,place_name,place_website,place_rating,review_mean,review_std,checkins,users,place_website_4q,group_id,group_followers_count,world_rank,country_rank,link_numbers,distance_metro,district
0,55.664012,37.522861,ChIJxUJiGhlNtUYRjyucIeHbeVI,"Родильный дом №4, филиал №1 ГКБ им. В.В. Виног...",http://www.roddom4.ru/,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1486796.0,102707.0,65.0,2022.568885,"Россия, Москва, Юго-Западный административный ..."
1,55.756778,37.638748,ChIJB437ifVKtUYR7EMUU6VApLg,Национальный медицинский исследовательский цен...,http://www.gnicpm.ru/,4.3,0.25,0.433013,67.0,31.0,https://www.gnicpm.ru,NaN,NaN,556465.0,38222.0,281.0,582.634054,"Россия, Москва, Центральный административный о..."
2,55.755245,37.656547,ChIJ2acH14xKtUYRCFp01z3Cgbk,МедЦентрСервис,http://medtsentrservis.ru/,4.7,NaN,NaN,NaN,NaN,NaN,100098038.0,474.0,10188597.0,NaN,0.0,308.311597,"Россия, Москва, Центральный административный о..."


## Domofond - загрузим данные о платежеспособности района ЛУПов

In [261]:
def table_to_df(table_body):
    
    '''
    
    Функция для перевода таблицы в pandas.DataFrame()
    
    params table_body: таблица
    
    '''
    
    data = []
    rows = table_body.find_all('tr')
    for row in rows:
        try:
            link = row['data-url']
        except:
            link = ''
            continue
        finally: 
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            cols.append(link)
            data.append(cols)

    return pd.DataFrame(data)


def parse_domofond(city):
    
    '''
    
    Функция для загрузки рейтинга районов городов России
    
    param city: город
    
    
    '''
    
    homepage = 'https://www.domofond.ru'
    
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
          }

        
    url = homepage + '/city-ratings'
    r = requests.get(url, headers = headers)
    soup = BeautifulSoup(r.text.encode('utf8'), "lxml")

    df_cities = table_to_df(soup.findAll('div', {'id': 'cityDistricts'})[0])

    
    if len(df_cities[df_cities[0] == city]) != 0:
        url_city = homepage + df_cities[df_cities[0] == city][12].values[0]
        r = requests.get(url_city, headers = headers)
        soup = BeautifulSoup(r.text.encode('utf8'), "lxml")

        table = soup.findAll('div', {'id': 'cityDistricts'})

        df_districts = table_to_df(soup.findAll('div', {'id': 'cityDistricts'})[0])
        df_districts.columns = ['district', 'rating']

        return df_districts[[0, 11]].iloc[1:]
    
    else:
        return None

In [264]:
# загрузим районы Москвы
moscow_district_df = parse_domofond("Москва")
moscow_district_df.head(3)

,district,rating
1,Куркино,1
2,Митино,4
3,Крылатское,72


In [304]:
# обоготим название района

for i, row in moscow_district_df.iterrows():
    
    a = row['district']
    r = requests.get('https://geocode-maps.yandex.ru/1.x/?format=json&geocode=' + a)
    r.enconding = 'cp1251'
    j = json.loads(r.text)
    
    ar = j.get('response').get('GeoObjectCollection').get('featureMember')
    if len(ar) != 0:
        
        lat, long = ar[0].get('GeoObject').get('Point').get('pos').split(' ')
        lat, long = float(lat), float(long)

        moscow_district_df.loc[i, 'lat'] = lat
        moscow_district_df.loc[i, 'long'] = long
        
        r = requests.get('https://geocode-maps.yandex.ru/1.x/?format=json&geocode=' + ','.join([str(lat), str(long)]) + '&kind=district' + '&spn=0.552069,0.400552')
        r.enconding = 'cp1251'
        j = json.loads(r.text)
        ar = j.get('response').get('GeoObjectCollection').get('featureMember')
        if len(ar) != 0 and ar[0] is not None:
            try: 
                moscow_district_df.loc[i, 'district_ya'] = ar[0].get('GeoObject').get('metaDataProperty').get('GeocoderMetaData').get('text')
            except AttributeError:
                continue
        

In [303]:
# добавим рейтинг районов в данные

data = data.merge(moscow_district_df[['rating', 'district_ya']].rename(columns={'district_ya':'district'}), on = ['district'], how = 'left')
data.loc[data[data.duplicated('place_id', keep=False)].index, 'rating'] = None
data.drop_duplicates(inplace = True)
data.head(3)

,lat,long,place_id,place_name,place_website,place_rating,review_mean,review_std,checkins,users,place_website_4q,group_id,group_followers_count,world_rank,country_rank,link_numbers,distance_metro,district,rating
0,55.664012,37.522861,ChIJxUJiGhlNtUYRjyucIeHbeVI,"Родильный дом №4, филиал №1 ГКБ им. В.В. Виног...",http://www.roddom4.ru/,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1486796.0,102707.0,65.0,2022.568885,"Россия, Москва, Юго-Западный административный ...",56
1,55.756778,37.638748,ChIJB437ifVKtUYR7EMUU6VApLg,Национальный медицинский исследовательский цен...,http://www.gnicpm.ru/,4.3,0.25,0.433013,67.0,31.0,https://www.gnicpm.ru,NaN,NaN,556465.0,38222.0,281.0,582.634054,"Россия, Москва, Центральный административный о...",100
2,55.755245,37.656547,ChIJ2acH14xKtUYRCFp01z3Cgbk,МедЦентрСервис,http://medtsentrservis.ru/,4.7,NaN,NaN,NaN,NaN,NaN,100098038.0,474.0,10188597.0,NaN,0.0,308.311597,"Россия, Москва, Центральный административный о...",100


## Save

In [316]:
data.to_csv('data/data_perpared.csv', encoding='cp1251', sep=';', index = False)